In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from ppxf_functions_Z import (
    load_spectrum,
    process_spectrum,
    rebin_to_log,
    make_noise,
    calculate_velscale_fwhm,
    run_ppxf,
    plot_ppxf,
)

In [2]:
# --- Inputs ---
galaxy_files = [
    (f"../calibrated data/SB{i}_data_orth.npy", f"../calibrated data/SB{i}_waveobs_orth.npy")
    for i in range(1, 8)
]

# Match each dataset to its redshift (same order)
redshifts = [
    0.017206345, 
    0.038136434,
    0.041185744,
    0.045207640,
    0.042784720,
    0.017624998,
    0.046458002
]


In [3]:
window_size = 10
step        = 1
scale       = 0.4   # arcsec per pixel
pix_min, pix_max = 150, 250

all_results = {}

# --- Loop over galaxies ---
for (data_file, wave_file), z in zip(galaxy_files, redshifts):

    print(f"\nRunning galaxy {data_file} with z={z}")

    raw = np.load(data_file)       # flux × spatial pixel
    waveem = np.load(wave_file)

    results = []
    raw_roi = raw[:, pix_min:pix_max]

    for start in range(0, raw_roi.shape[1] - window_size + 1, step):
        end = start + window_size
        spectrum = raw_roi[:, start:end].sum(axis=1)

        # Build DataFrame
        df_spec = pd.DataFrame({
            "waveem": waveem,
            "flux":   spectrum
        })

        # Process, rebin, noise
        df_cut = process_spectrum(df_spec)
        df_rb, lam, flux_rb, ln_w = rebin_to_log(df_cut)

        noise = make_noise(df_rb)
        velscale, fwhm = calculate_velscale_fwhm(ln_w, lam)
        center = start + window_size//2 + pix_min

        # Run pPXF with galaxy's redshift
        pp = run_ppxf(lam, fwhm, velscale, df_rb, noise, redshift=z)

        print(f'  PIXEL: {center}')

        # Store result
        results.append((center, df_rb, pp))

    # Save per galaxy
    all_results[data_file] = results


Running galaxy ../calibrated data/SB1_data_orth.npy with z=0.017206345
Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      4756      1000
 comp.  1:      5396       222
chi2/DOF: 0.7400; DOF: 1586; degree = 4; mdegree = 10
method = capfit; Jac calls: 13; Func calls: 197; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 4/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha     0.4635       1.4    5396   222
Comp:  1             [SII]6716      3.103       1.4    5396   222
Comp:  1             [SII]6731     0.3051       1.4    5396   222
Comp:  1           [NII]6583_d      0.000       1.7    5396   222
-------------

 Best Fit:       Vel     sigma
 comp.  0:      5032      1000
 comp.  1:      5558       182
chi2/DOF: 0.6988; DOF: 1586; degree = 4; mdegree = 10
method = capfit; Jac calls: 12; Func calls: 187; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      5.437       1.0    5558   182
Comp:  1             [SII]6716      1.905       1.1    5558   182
Comp:  1             [SII]6731      1.657       1.1    5558   182
Comp:  1           [NII]6583_d     0.9219       1.3    5558   182
-----------------------------------------------------------------
  PIXEL: 163
Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best 

Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      4281       102
 comp.  1:      4735       167
chi2/DOF: 0.2471; DOF: 1586; degree = 4; mdegree = 10
method = capfit; Jac calls: 17; Func calls: 260; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 6/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      3.335      0.86    4735   167
Comp:  1             [SII]6716      0.000      0.87    4735   167
Comp:  1             [SII]6731      2.056      0.87    4735   167
Comp:  1           [NII]6583_d      5.908       1.1    4735   167
-----------------------------------------------------------------
  PIXEL: 172
Emissi

Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      4452       454
 comp.  1:      5218       285
chi2/DOF: 0.6522; DOF: 1586; degree = 4; mdegree = 10
method = capfit; Jac calls: 13; Func calls: 203; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 4/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      1.314      0.65    5218   285
Comp:  1             [SII]6716      1.494      0.68    5218   285
Comp:  1             [SII]6731      0.000      0.68    5218   285
Comp:  1           [NII]6583_d      2.404      0.82    5218   285
-----------------------------------------------------------------
  PIXEL: 181
Emissi

Emission lines included in gas templates:
['Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[SII]6716' '[SII]6731'
 '[NeIII]3968' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:      4710       341
 comp.  1:      4816       254
chi2/DOF: 0.01916; DOF: 1586; degree = 4; mdegree = 10
method = capfit; Jac calls: 7; Func calls: 107; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 7/154
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      12.25      0.76    4816   254
Comp:  1             [SII]6716      1.837      0.79    4816   254
Comp:  1             [SII]6731      1.869      0.79    4816   254
Comp:  1           [NII]6583_d      8.393      0.96    4816   254
-----------------------------------------------------------------
  PIXEL: 190
Emissi

...

Emission lines included in gas templates:
['H9' 'H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     13614       120
 comp.  1:     13614        50
chi2/DOF: 0.9589; DOF: 1599; degree = 4; mdegree = 10
method = capfit; Jac calls: 1; Func calls: 21; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 1/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      0.000       4.8   13614    50
Comp:  1           [NII]6583_d      3.367       6.1   13614    50
-----------------------------------------------------------------
  PIXEL: 228
Emission lines included in gas templates:
['H9' 'H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5

Emission lines included in gas templates:
['H9' 'H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     13569        63
 comp.  1:     13683       131
chi2/DOF: 0.9831; DOF: 1588; degree = 4; mdegree = 10
method = capfit; Jac calls: 6; Func calls: 95; Status: 2
linear_method = lsq_box; Nonzero Templates (>0.1%): 3/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      2.944       13.   13683   131
Comp:  1           [NII]6583_d      2.582       17.   13683   131
-----------------------------------------------------------------
  PIXEL: 238
Emission lines included in gas templates:
['H9' 'H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5

/Users/owner1/anaconda3/lib/python3.11/site-packages/capfit/capfit.py:799: RuntimeWarning: invalid value encountered in scalar divide
  ratio = actred/prered


Emission lines included in gas templates:
['H9' 'H8' 'Heps' 'Hdelta' 'Hgamma' 'Hbeta' 'Halpha' '[NeIII]3968'
 '[NeIII]3869' 'HeII4687' 'HeI5876' '[OIII]5007_d' '[OI]6300_d'
 '[NII]6583_d']
 Best Fit:       Vel     sigma
 comp.  0:     13614       120
 comp.  1:     13614        50
chi2/DOF: 0.9949; DOF: 1602; degree = 4; mdegree = 10
method = capfit; Jac calls: 1; Func calls: 17; Status: 4
linear_method = lsq_box; Nonzero Templates (>0.1%): 0/152
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
gas_component           name        flux       err      V     sig
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Comp:  1                Halpha      0.000       32.   13614    50
Comp:  1           [NII]6583_d      0.000       41.   13614    50
-----------------------------------------------------------------
  PIXEL: 245


/Users/owner1/anaconda3/lib/python3.11/site-packages/capfit/capfit.py:799: RuntimeWarning: invalid value encountered in scalar divide
  ratio = actred/prered


In [4]:
import os
save_dir = "../halpha_results_Z"
os.makedirs(save_dir, exist_ok=True)

for data_file, results in all_results.items():
    galaxy_name = os.path.basename(data_file).replace("_data_orth.npy", "")

    centers, velocities, v_errs, dispersions, sigma_errs, EWs, EW_errs = [], [], [], [], [], [], []

    for center, df_rb, pp in results:
        # --- extract Hα kinematics ---
        sol_gas = pp.sol[1]
        err_gas = pp.error[1]
        v_ha, sigma_ha = sol_gas
        v_ha_err, sigma_ha_err = err_gas

        # Hα index in gas templates
        idx = np.where(pp.gas_names == "Halpha")[0][0]

        flux_ha   = pp.gas_flux[idx]
        continuum = pp.bestfit - pp.gas_bestfit
        peak_i    = np.argmax(pp.gas_bestfit_templates[:, idx])
        cont0     = continuum[peak_i]
        cont_err  = pp.noise[peak_i]
        flux_err  = pp.gas_flux_error[idx]

        EW     = flux_ha / cont0
        EW_err = abs(EW) * np.sqrt((cont_err/cont0)**2 + (flux_err/flux_ha)**2)

        # store
        centers.append(center)
        velocities.append(v_ha)
        v_errs.append(v_ha_err)
        dispersions.append(sigma_ha)
        sigma_errs.append(sigma_ha_err)
        EWs.append(EW)
        EW_errs.append(EW_err)

    # Build DataFrame
    df = pd.DataFrame({
        "center":        centers,
        "velocity":      velocities,
        "velocity_err":  v_errs,
        "dispersion":    dispersions,
        "dispersion_err": sigma_errs,
        "EW":            EWs,
        "EW_err":        EW_errs
    })

    # Save one CSV per galaxy
    out_path = os.path.join(save_dir, f"{galaxy_name}_halpha_results_orth.csv")
    df.to_csv(out_path, index=False)
    print(f"Saved {out_path}")


/var/folders/r3/l94405xd0mq2s62v2swzjq2m0000gn/T/ipykernel_5199/1628438408.py:28: RuntimeWarning: divide by zero encountered in scalar divide
  EW_err = abs(EW) * np.sqrt((cont_err/cont0)**2 + (flux_err/flux_ha)**2)
/var/folders/r3/l94405xd0mq2s62v2swzjq2m0000gn/T/ipykernel_5199/1628438408.py:28: RuntimeWarning: invalid value encountered in scalar multiply
  EW_err = abs(EW) * np.sqrt((cont_err/cont0)**2 + (flux_err/flux_ha)**2)


Saved ../halpha_results_Z/SB1_data_orth.npy_orth_halpha_results.csv
Saved ../halpha_results_Z/SB2_data_orth.npy_orth_halpha_results.csv
Saved ../halpha_results_Z/SB3_data_orth.npy_orth_halpha_results.csv
Saved ../halpha_results_Z/SB4_data_orth.npy_orth_halpha_results.csv
Saved ../halpha_results_Z/SB5_data_orth.npy_orth_halpha_results.csv
Saved ../halpha_results_Z/SB6_data_orth.npy_orth_halpha_results.csv
Saved ../halpha_results_Z/SB7_data_orth.npy_orth_halpha_results.csv
